In [1]:
import os
import datetime
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from itertools import chain
from selenium.webdriver.firefox.options import Options

In [2]:
urls = {
    'switzerland' : {
        '2022' : 'https://www.coachcox.co.uk/imstats/race/1887/results/',
        '2021' : 'https://www.coachcox.co.uk/imstats/race/1793/results/',
        '2019' : 'https://www.coachcox.co.uk/imstats/race/485/results/',
        '2018' : 'https://www.coachcox.co.uk/imstats/race/437/results/'
    },
    'copenhagen' : {
        '2022' : 'https://www.coachcox.co.uk/imstats/race/1880/results/',
        '2021' : 'https://www.coachcox.co.uk/imstats/race/1776/results/', 
        '2019' : 'https://www.coachcox.co.uk/imstats/race/492/results/', 
        '2018' : 'https://www.coachcox.co.uk/imstats/race/443/results/'
    }
}

In [3]:
def scrape(url):
    """
    Function takes a URL from www.coachcox.co.uk and scrapes the results data. Returns a DF containing the results 
    """
    # Set up webdriver and get the content
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    html = driver.page_source

    # Parse the html and locate the results table
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', id='imraceresultstable')

    # Grab the results and column titles from the table
    data = []
    labels = []
    for row in table.find_all('tr'):
        data_cells = row.find_all('td')
        data.append([cell.text for cell in data_cells])

        lebel_cells = row.find_all('th')
        labels.append([cell.text for cell in lebel_cells])

    # Flatten the list of labels
    labels = list(chain.from_iterable(labels))

    return pd.DataFrame(data, columns=labels)

In [4]:
def scrape_all_to_file(urls):
    for country, dates in urls.items():
        for date, url in dates.items(): 
            filename = f"dataFiles/{country}_{date}.res"
            df = scrape(url)
            df.to_pickle(filename)


In [5]:
# scrape_all_to_file(urls)

In [6]:
dataFiles = os.listdir("dataFiles")
dataFiles

['copenhagen_2018.res',
 'copenhagen_2019.res',
 'switzerland_2019.res',
 'switzerland_2018.res',
 'copenhagen_2022.res',
 'switzerland_2022.res',
 'switzerland_2021.res',
 'copenhagen_2021.res']

In [7]:
df = pd.read_pickle('dataFiles/copenhagen_2022.res')

In [8]:
# Remove top row (contains Nones)
df = df[1:]
# Put swim times in consistant format 
df['Swim Time'] = df['Swim Time'].apply(lambda x: '00:' + x if len(x.split(':')) < 3 else x)

,Bib,Name,Country,Gender,Division,Division Rank,Overall Time,Overall Rank,Swim Time,Swim Rank,Bike Time,Bike Rank,Run Time,Run Rank,Finish Status
1,2141,Benjamin Winkler,Germany,Male,M35-39,1,8:19:20,1,00:54:25,25,4:29:27,13,2:49:29,3,Finisher
2,73,Lars Petter Stormo,Norway,Male,M40-44,1,8:27:43,2,00:53:55,20,4:31:03,20,2:57:34,7,Finisher
3,667,Chris Weeks,United Kingdom,Male,M30-34,1,8:31:26,3,00:57:16,79,4:32:06,24,2:54:59,4,Finisher
4,12,Lee Williams,United Kingdom,Male,M40-44,2,8:31:26,4,00:53:58,21,4:30:47,18,3:01:04,10,Finisher
5,869,Alexander Beck,Germany,Male,M30-34,2,8:32:33,5,00:59:18,128,4:25:56,5,2:55:59,5,Finisher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,2320,Denis Berckmans,Belgium,Male,M45-49,,,,1:19:37,1436,7:27:47,2132,,,DNF
2333,2131,Lício Eduardo Oliveira,Brazil,Male,M40-44,,,,1:20:29,1511,,,,,DNF
2334,1991,Julie Niewald Anhøj,Denmark,Female,F25-29,,,,1:36:44,2070,,,,,DNF
2335,1413,James Gunn,United Kingdom,Male,M45-49,,,,2:09:21,2195,,,,,DNF


In [10]:
# Convert time columns to duration
df['Swim Time'] = pd.to_timedelta(df['Swim Time'],errors='coerce')
df['Bike Time'] = pd.to_timedelta(df['Bike Time'])
df['Run Time'] = pd.to_timedelta(df['Run Time'])


In [14]:
df

,Bib,Name,Country,Gender,Division,Division Rank,Overall Time,Overall Rank,Swim Time,Swim Rank,Bike Time,Bike Rank,Run Time,Run Rank,Finish Status
1,2141,Benjamin Winkler,Germany,Male,M35-39,1,8:19:20,1,0 days 00:54:25,25,0 days 04:29:27,13,0 days 02:49:29,3,Finisher
2,73,Lars Petter Stormo,Norway,Male,M40-44,1,8:27:43,2,0 days 00:53:55,20,0 days 04:31:03,20,0 days 02:57:34,7,Finisher
3,667,Chris Weeks,United Kingdom,Male,M30-34,1,8:31:26,3,0 days 00:57:16,79,0 days 04:32:06,24,0 days 02:54:59,4,Finisher
4,12,Lee Williams,United Kingdom,Male,M40-44,2,8:31:26,4,0 days 00:53:58,21,0 days 04:30:47,18,0 days 03:01:04,10,Finisher
5,869,Alexander Beck,Germany,Male,M30-34,2,8:32:33,5,0 days 00:59:18,128,0 days 04:25:56,5,0 days 02:55:59,5,Finisher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,2320,Denis Berckmans,Belgium,Male,M45-49,,,,0 days 01:19:37,1436,0 days 07:27:47,2132,NaT,,DNF
2333,2131,Lício Eduardo Oliveira,Brazil,Male,M40-44,,,,0 days 01:20:29,1511,NaT,,NaT,,DNF
2334,1991,Julie Niewald Anhøj,Denmark,Female,F25-29,,,,0 days 01:36:44,2070,NaT,,NaT,,DNF
2335,1413,James Gunn,United Kingdom,Male,M45-49,,,,0 days 02:09:21,2195,NaT,,NaT,,DNF


In [ ]:
wardy = df[df.apply(lambda r: r.str.contains('Medhurst', case=False).any(), axis=1)] 

In [ ]:
def bike_speed(bike_time):
    bike_distance = 180.2 # 112mile, 180.2km 

    
